##  [망고플레이트 크롤링](https://www.mangoplate.com/)    
- **데이터 정제(Data Cleansing)**   
- 참고 사이트 : https://juwon2021.tistory.com/414

### ** 시작 전 확인 사항 **
  Window terminal에서   
- conda activate   
- conda activate ml-env   
(ml-env) 환경 확인 후   
- jupyter notebook   
- conda install -c anaconda requests   
- conda install bs4   
- conda install lxml   
- conda install -c conda-forge selenium   
- conda install tqdm   
- connda install -c conda-forge folium   
- connda install -c conda-forge googlemaps   
- pip install chromedriver-autoinstaller   

* * *

### 데이터 정제(Data Cleansing)

In [1]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.keys import Keys

In [2]:
# 저장한 csv 불러오기
df1 = pd.read_csv('data/Gangnam1.csv')
df2 = pd.read_csv('data/Gangnam2.csv')
df3 = pd.read_csv('data/Gangnam3.csv')
df4 = pd.read_csv('data/Gangnam4.csv')
df5 = pd.read_csv('data/Gangnam5.csv')
df6 = pd.read_csv('data/Gangnam6.csv')
df7 = pd.read_csv('data/Gangnam7.csv')
df8 = pd.read_csv('data/Gangnam8.csv')
df9 = pd.read_csv('data/Gangnam9.csv')
df10 = pd.read_csv('data/Gangnam10.csv')

In [3]:
df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]
df_all = pd.concat(df_list, ignore_index=True)
df_all

,title,Point,View,Review,Star,Type
0,미라이,4.8,"510,477",127,"10,338","['이자카야 / 오뎅 / 꼬치', '사시미8종 (2인)\n41,000원\n마구로치즈..."
1,시라카와,4.8,"321,374",75,"6,227",이자카야 / 오뎅 / 꼬치
2,페리지,4.8,"52,571",39,"1,467","['기타 양식', '페리지 에끌래어\n8,000원\n봉골레\n31,000원\n아뇰로..."
3,상진식감,4.8,"30,936",41,878,까스 요리
4,맛짱조개,4.8,"130,259",64,"3,855","['해산물 요리', '조개찜 (중)\n48,000원\n해물모듬\n35,000원\n가..."
...,...,...,...,...,...,...
195,고료리 켄,4.3,"94,597",22,"2,458",정통 일식 / 일반 일식
196,쁠로13,4.3,"87,215",81,"3,159",베이커리
197,치즈룸,4.3,"63,261",62,"1,719",이탈리안
198,쎄쎄종,4.3,"59,705",58,"2,246","['카페 / 디저트', '아메리카노\n4,500원\n카페라떼\n5,000원\n오늘의..."


In [4]:
# csv 파일 저장
df_all.to_csv('Gangnam_Mango.csv', encoding='utf-8-sig', index=False)

* * *

### 결측치 존재 확인

In [5]:
# 파일 불러오기
df_all = pd.read_csv('data/Gangnam_Mango.csv')

In [6]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   200 non-null    object 
 1   Point   200 non-null    float64
 2   View    200 non-null    object 
 3   Review  200 non-null    int64  
 4   Star    200 non-null    object 
 5   Type    200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [7]:
### 열 이름 수정
# 망고 데이터 이름 통일
df_all.rename(columns = {'title' : 'Title'}, inplace=True)
print(df_all.columns)

Index(['Title', 'Point', 'View', 'Review', 'Star', 'Type'], dtype='object')


* * *

### type 열 정제

In [8]:
# 한글만 추출
data = df_all.copy()
# data['Type'] = data['Type'].str.findall('[a-z|A-Z|0-9|가-힣]+')
data['Type'] = data['Type'].str.findall('[가-힣]+')
data['Type']

0      [이자카야, 오뎅, 꼬치, 사시미, 종, 인, 원, 마구로치즈아에, 원, 양갈비, ...
1                                         [이자카야, 오뎅, 꼬치]
2      [기타, 양식, 페리지, 에끌래어, 원, 봉골레, 원, 아뇰로띠, 원, 안다리노, ...
3                                               [까스, 요리]
4      [해산물, 요리, 조개찜, 중, 원, 해물모듬, 원, 가리비숙회, 원, 포뜬산오징어...
                             ...                        
195                                     [정통, 일식, 일반, 일식]
196                                               [베이커리]
197                                               [이탈리안]
198    [카페, 디저트, 아메리카노, 원, 카페라떼, 원, 오늘의, 밀크티, 원, 바닐라타...
199         [한정식, 백반, 정통, 한식, 고기만두, 원, 김치만두, 원, 새우만두, 원]
Name: Type, Length: 200, dtype: object

In [9]:
# 공백문자로 word 단위로 문자열로 반환
data['Type'] = data['Type'].apply(lambda x : (' ').join(x))
data['Type']

0      이자카야 오뎅 꼬치 사시미 종 인 원 마구로치즈아에 원 양갈비 스미비 야끼 원 노도...
1                                             이자카야 오뎅 꼬치
2      기타 양식 페리지 에끌래어 원 봉골레 원 아뇰로띠 원 안다리노 윗 랍스타 원 텐더로인 원
3                                                  까스 요리
4          해산물 요리 조개찜 중 원 해물모듬 원 가리비숙회 원 포뜬산오징어 원 해물라면 원
                             ...                        
195                                          정통 일식 일반 일식
196                                                 베이커리
197                                                 이탈리안
198      카페 디저트 아메리카노 원 카페라떼 원 오늘의 밀크티 원 바닐라타르트 원 코코오레 원
199                    한정식 백반 정통 한식 고기만두 원 김치만두 원 새우만두 원
Name: Type, Length: 200, dtype: object

In [10]:
# '원' 문자 제거
data['Type'] = data['Type'].str.replace('원', '')
data['Type']

0      이자카야 오뎅 꼬치 사시미 종 인  마구로치즈아에  양갈비 스미비 야끼  노도구로 ...
1                                             이자카야 오뎅 꼬치
2           기타 양식 페리지 에끌래어  봉골레  아뇰로띠  안다리노 윗 랍스타  텐더로인 
3                                                  까스 요리
4               해산물 요리 조개찜 중  해물모듬  가리비숙회  포뜬산오징어  해물라면 
                             ...                        
195                                          정통 일식 일반 일식
196                                                 베이커리
197                                                 이탈리안
198           카페 디저트 아메리카노  카페라떼  오늘의 밀크티  바닐라타르트  코코오레 
199                       한정식 백반 정통 한식 고기만두  김치만두  새우만두 
Name: Type, Length: 200, dtype: object

* * *

### 문자열에서 숫자형으로 변환 : View, Star 열

- View 열

In [11]:
# 모든 원소들을 문자형으로 반환
data['View'] = [str(x) for x in data['View']]
data['View'] = data['View'].str.replace(',', '')
print(data['View'].isna().sum(), '\n')
print(data['View'])

0 

0      510477
1      321374
2       52571
3       30936
4      130259
        ...  
195     94597
196     87215
197     63261
198     59705
199    346218
Name: View, Length: 200, dtype: object


In [12]:
data['View'] = data['View'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Title   200 non-null    object 
 1   Point   200 non-null    float64
 2   View    200 non-null    int32  
 3   Review  200 non-null    int64  
 4   Star    200 non-null    object 
 5   Type    200 non-null    object 
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 8.7+ KB


- Star 열

In [13]:
# 모든 원소들을 문자형으로 반환
star_list = [str(x) for x in data['Star']]
data['Star'] = star_list
data['Star']

0      10,338
1       6,227
2       1,467
3         878
4       3,855
        ...  
195     2,458
196     3,159
197     1,719
198     2,246
199     8,613
Name: Star, Length: 200, dtype: object

In [14]:
data['Star'] = data['Star'].str.replace(',', '')
print(data['Star'].isna().sum(), '\n')
print(data['Star'])

0 

0      10338
1       6227
2       1467
3        878
4       3855
       ...  
195     2458
196     3159
197     1719
198     2246
199     8613
Name: Star, Length: 200, dtype: object


In [16]:
data['Star'] = data['Star'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Title   200 non-null    object 
 1   Point   200 non-null    float64
 2   View    200 non-null    int32  
 3   Review  200 non-null    int64  
 4   Star    200 non-null    int32  
 5   Type    200 non-null    object 
dtypes: float64(1), int32(2), int64(1), object(2)
memory usage: 7.9+ KB


In [17]:
#정제 완료한 데이터를 csv로 저장
data.to_csv('mango_revise.csv', encoding='utf-8-sig', index=False)